In [1]:
import pandas as pd
import random
import numpy as np
import implicit
from sklearn.model_selection import train_test_split
from pandas.api.types import CategoricalDtype
from scipy.sparse import csr_matrix, save_npz


In [2]:
restaurant_df = pd.read_csv("../data/clean/restaurant.csv")

In [15]:
reviews_df = pd.read_csv("../data/clean/review.csv")

In [16]:
reviews_df.shape
reviews_df.head()

,business_id,name,user_id,stars
0,MTSW4McQd7CbVtyjqoe9mw,st honore pastries,6_SpY41LIHZuIaiDs5FMKA,4
1,MTSW4McQd7CbVtyjqoe9mw,st honore pastries,tCXElwhzekJEH6QJe3xs7Q,4
2,MTSW4McQd7CbVtyjqoe9mw,st honore pastries,WqfKtI-aGMmvbA9pPUxNQQ,5
3,MTSW4McQd7CbVtyjqoe9mw,st honore pastries,3-1va0IQfK-9tUMzfHWfTA,5
4,MTSW4McQd7CbVtyjqoe9mw,st honore pastries,EouCKoDfzaVG0klEgdDvCQ,4


In [17]:
unique_users = list(reviews_df['user_id'].unique())
unique_restaurant = list(reviews_df['business_id'].unique())
rating = reviews_df['stars'].tolist()

# converting users and restaurants into numerical ids
rows = reviews_df.user_id.astype(CategoricalDtype(categories=unique_users)).cat.codes
cols = reviews_df.business_id.astype(CategoricalDtype(categories=unique_restaurant)).cat.codes

reviews_df['users_id_code'] = rows
reviews_df['business_id_code'] = cols
sparse_restaurant_user = csr_matrix((rating, (cols, rows)), shape=(len(unique_restaurant), len(unique_users)))
sparse_user_restaurant = csr_matrix((rating, (rows, cols)), shape=(len(unique_users), len(unique_restaurant)))

In [19]:
ratings = sparse_user_restaurant
test_set = ratings.copy()
test_set[test_set != 0] = 1
training_set = ratings.copy()
user_restaurant_interaction = training_set.nonzero()
interaction_index_pair = list(zip(user_restaurant_interaction[0], user_restaurant_interaction[1]))
random.seed(0)
test_set_size = int(np.ceil(0.2 * len(interaction_index_pair)))
test_samples = random.sample(interaction_index_pair, test_set_size)
user_index = [index[0] for index in test_samples]
restaurant_index = [index[1] for index in test_samples]
training_set[user_index, restaurant_index] = 0
training_set.eliminate_zeros()

In [46]:
als_model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)
als_model.fit(train_data.T)

In [ ]:
reviews_df[reviews_df[]]

In [47]:
user_id = 5858
ids, scores = als_model.recommend(user_id, sparse_user_restaurant[user_id])
restaurant = []
for id in ids:
    restaurant.append(reviews_df.name.loc[reviews_df.business_id_code == id].iloc[0])
print("Recommendation For User", )
pd.DataFrame(restaurant)

IndexError: index 5858 is out of bounds for axis 0 with size 5857

In [37]:
train_data.shape

(209582, 5857)

In [44]:
business_id = 12
ids, scores = als_model.similar_items(business_id)
restaurant = []
for id in ids:
    restaurant.append(reviews_df.name.loc[reviews_df.business_id_code == id].iloc[0])
print("Restaurants Similar to", reviews_df.name.loc[reviews_df.business_id_code == id].iloc[0])
pd.DataFrame(restaurant)

Restaurants Similar to big geez chicken shack


,0
0,baltic bakery
1,village bar and kitchen
2,first wok
3,talula's garden
4,minas pizza
5,hibachi grill & supreme buffet
6,52nd skyline restaurant
7,street side
8,lascala's
9,big geez chicken shack
